In [2]:
import sys
import os
import pathlib
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from dotenv import load_dotenv

# import openai
# let's use Google Gemini
import google.generativeai as genai


# load API keys
load_dotenv()
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [3]:
nse_df = pd.read_csv("nse.csv")
nse_df.head()

,Symbol,Company
0,ADANIENT.NS,Adani Enterprises Ltd
1,ADANIPORTS.NS,Adani Ports and Special Economic Zone Ltd
2,APOLLOHOSP.NS,Apollo Hospitals Enterprise Ltd.
3,ASIANPAINT.NS,Asian Paints Ltd
4,AXISBANK.NS,Axis Bank Ltd


In [4]:
symbols = list(nse_df["Symbol"])

In [5]:
def get_stock_fundamentals(symbol):
    try:
        stock = yf.Ticker(symbol)
        info = stock.info

        # Extract Key Metrics
        data = {
            # "Symbol": symbol,
            "Company": info.get("longName", "N/A"),
            "Market Cap (Cr)": info.get("marketCap", 0) / 1e7,  # Convert to Crore
            "P/E Ratio": info.get("trailingPE", np.nan),
            "P/B Ratio": info.get("priceToBook", np.nan),
            "ROE (%)": (
                info.get("returnOnEquity", 0) * 100
                if info.get("returnOnEquity")
                else np.nan
            ),
            "ROCE (%)": (
                info.get("returnOnAssets", 0) * 100
                if info.get("returnOnAssets")
                else np.nan
            ),  # Proxy for ROCE
            "Debt/Equity": info.get("debtToEquity", np.nan),
            "Profit Margin (%)": (
                info.get("profitMargins", 0) * 100
                if info.get("profitMargins")
                else np.nan
            ),
            "Revenue Growth (YoY %)": (
                info.get("revenueGrowth", 0) * 100
                if info.get("revenueGrowth")
                else np.nan
            ),
            "FCF (Cr)": (
                info.get("freeCashflow", 0) / 1e7
                if info.get("freeCashflow")
                else np.nan
            ),  # Convert to Crore
            "Dividend Yield (%)": (
                info.get("dividendYield", 0) * 100
                if info.get("dividendYield")
                else np.nan
            ),
            "EV/EBITDA": info.get("enterpriseToEbitda", np.nan),
        }
        return data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

In [6]:
nse_stocks = pd.DataFrame()

for symbol in symbols:
    nse_stocks[symbol] = get_stock_fundamentals(symbol)

# transpose such that stock symbol becomes the rows
nse_stocks = nse_stocks.T

In [7]:
# Define thresholds for selection
def evaluate_stock(stock):
    if (
        stock["P/E Ratio"] < 20
        and stock["P/B Ratio"] < 3
        and stock["ROE (%)"] > 15
        and stock["ROCE (%)"] > 12
        and stock["Debt/Equity"] < 1
        and stock["Profit Margin (%)"] > 10
        and stock["Revenue Growth (YoY %)"] > 10
        and stock["FCF (Cr)"] > 0
        and stock["EV/EBITDA"] < 12
    ):
        return "✅ Strong Buy"
    elif stock["P/E Ratio"] < 25 and stock["ROE (%)"] > 12:
        return "🔎 Moderate Buy"
    else:
        return "❌ Avoid"

In [20]:
# Use OpenAI to generate recommendations
def get_ai_recommendation(stock):
    prompt = f"""
    You are a Financial Investments Expert.
    Analyze the following stock based on key fundamental metrics and determine if it is a good long-term investment:

    Company: {stock['Company']}
    Market Cap: {stock['Market Cap (Cr)']} Cr
    P/E Ratio: {stock['P/E Ratio']}
    P/B Ratio: {stock['P/B Ratio']}
    ROE: {stock['ROE (%)']}%
    ROCE: {stock['ROCE (%)']}%
    Debt-to-Equity: {stock['Debt/Equity']}
    Profit Margin: {stock['Profit Margin (%)']}%
    Revenue Growth: {stock['Revenue Growth (YoY %)']}%
    Free Cash Flow: {stock['FCF (Cr)']} Cr
    Dividend Yield: {stock['Dividend Yield (%)']}%
    EV/EBITDA: {stock['EV/EBITDA']}

    Based on these factors, provide a **detailed investment recommendation** stating whether it is a **buy, hold, or avoid**. Explain the reasons in an investor-friendly language.
    """
    print(prompt, flush=True)

    # response = openai.chat.completions.create(
    #     model="gpt-4",
    #     messages=[
    #         {"role": "system", "content": "You are a financial investment expert."},
    #         {"role": "user", "content": prompt},
    #     ],
    # )
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            max_output_tokens=2048,
            temperature=0.1,
        ),
    )

    # return response.choices[0].message
    return response.text

In [11]:
# using our evaluation criteria
nse_stocks["Investment Reco"] = nse_stocks.apply(evaluate_stock, axis=1)

In [21]:
response = get_ai_recommendation(nse_stocks.iloc[0])
print(response)


    You are a Financial Investments Expert.
    Analyze the following stock based on key fundamental metrics and determine if it is a good long-term investment:

    Company: Adani Enterprises Limited
    Market Cap: 260529.1667456 Cr
    P/E Ratio: 75.24191
    P/B Ratio: 7.052274
    ROE: 13.639999999999999%
    ROCE: 3.8089999999999997%
    Debt-to-Equity: 168.936
    Profit Margin: 5.41%
    Revenue Growth: 0.4%
    Free Cash Flow: -17090.0307968 Cr
    Dividend Yield: 0.059999997%
    EV/EBITDA: 26.405

    Based on these factors, provide a **detailed investment recommendation** stating whether it is a **buy, hold, or avoid**. Explain the reasons in an investor-friendly language.
    
Adani Enterprises Limited presents a complex investment case with significant red flags that strongly suggest an **avoid** recommendation for long-term investors.  Let's break down the key metrics:

**Negative Indicators:**

* **High P/E and P/B Ratios:** A P/E ratio of 75.24 and a P/B ratio of 7.05

In [16]:
# what does OpenAI say? Will take some time
nse_stocks["AI Reco"] = nse_stocks.apply(get_openai_recommendation, axis=1)

KeyboardInterrupt: 

In [31]:
# nse_stocks.loc[:, ["Company", "Investment Reco", "AI Reco"]]
nse_stocks.loc[:, ["Investment Reco", "AI Reco"]]

,Company,Investment Recommendation
ADANIENT.NS,Adani Enterprises Limited,❌ Avoid
ADANIPORTS.NS,Adani Ports and Special Economic Zone Limited,🔎 Moderate Buy
APOLLOHOSP.NS,Apollo Hospitals Enterprise Limited,❌ Avoid
ASIANPAINT.NS,Asian Paints Limited,❌ Avoid
AXISBANK.NS,Axis Bank Limited,🔎 Moderate Buy
BAJAJ-AUTO.NS,Bajaj Auto Limited,❌ Avoid
BAJFINANCE.NS,Bajaj Finance Limited,❌ Avoid
BAJAJFINSV.NS,Bajaj Finserv Ltd.,❌ Avoid
BEL.NS,Bharat Electronics Limited,❌ Avoid
BPCL.NS,Bharat Petroleum Corporation Limited,🔎 Moderate Buy
